<a href="https://colab.research.google.com/github/danie033/Extra_Code/blob/main/pyspark_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession,Row
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType,StructField,IntegerType,DoubleType,StringType


In [ ]:
spark=SparkSession.builder.appName('Read CSV file into DataFrame').getOrCreate()

data=spark.read.csv('/content/sample_data/Gaming_data.csv',sep=',',inferSchema=True,header=True)


In [ ]:
#df=data.toPandas()
#data.show()
#data2=data.withColumnRenamed("occupation","User_Occupation")
#data2.show()

In [ ]:
#data.select('Daily_gaming_hours').show()
data.head(3)
data.printSchema()

root
 |-- User_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Game_Type: string (nullable = true)
 |-- Daily_Gaming_Hours: double (nullable = true)
 |-- Weekly_Gaming_Hours: double (nullable = true)
 |-- Primary_Gaming_Time: string (nullable = true)
 |-- Sleep_Hours: double (nullable = true)
 |-- Stress_Level: integer (nullable = true)
 |-- Focus_Level: integer (nullable = true)
 |-- Academic_or_Work_Score: integer (nullable = true)
 |-- Productivity_Level: integer (nullable = true)
 |-- Performance_Impact: string (nullable = true)



**Altering columns**

Here I will continue using the pyspark dataframes to work with the currente data

In [ ]:

new_schema=StructType([
    StructField("age",IntegerType(),True),
    StructField("height",DoubleType(),True),
    StructField("Name",StringType(),True),
])

#Before:
# newData=spark.createDataFrame([
#     Row(age=10,height=12,Name='Alice'),
#     Row(age=3,height=12,Name='Danie;')
# ])

#data3.show()
#data3.printSchema()

newData=spark.createDataFrame([
    (None,12.4,"Daniel"),
    (30,12.4,"Omar"),
    (None,float('nan'),None),
    (15,float('nan'),"Danna"),
    (67,12.4,"Luis"),
    (None,23.,None),
    (15,12.4,"Danna"),
    (67,12.4,"Luis"),
    (None,float('nan'),None),
    (15,12.4,"Danna"),
    (67,12.4,"Luis"),
],schema=new_schema)

newData.show()
newData.printSchema()

+----+------+------+
| age|height|  Name|
+----+------+------+
|NULL|  12.4|Daniel|
|  30|  12.4|  Omar|
|NULL|   NaN|  NULL|
|  15|   NaN| Danna|
|  67|  12.4|  Luis|
|NULL|  23.0|  NULL|
|  15|  12.4| Danna|
|  67|  12.4|  Luis|
|NULL|   NaN|  NULL|
|  15|  12.4| Danna|
|  67|  12.4|  Luis|
+----+------+------+

root
 |-- age: integer (nullable = true)
 |-- height: double (nullable = true)
 |-- Name: string (nullable = true)



In [ ]:
newData2=newData.na.drop(subset="name")
newData2.show()

+----+------+------+
| age|height|  Name|
+----+------+------+
|NULL|  12.4|Daniel|
|  30|  12.4|  Omar|
|  15|   NaN| Danna|
|  67|  12.4|  Luis|
|  15|  12.4| Danna|
|  67|  12.4|  Luis|
|  15|  12.4| Danna|
|  67|  12.4|  Luis|
+----+------+------+



In [ ]:
newData3=newData2.na.fill({"age":100,"Height":5.8})
newData3.show()

+---+------+------+
|age|height|  Name|
+---+------+------+
|100|  12.4|Daniel|
| 30|  12.4|  Omar|
| 15|   5.8| Danna|
| 67|  12.4|  Luis|
| 15|  12.4| Danna|
| 67|  12.4|  Luis|
| 15|  12.4| Danna|
| 67|  12.4|  Luis|
+---+------+------+



In [ ]:
newData4=newData3.where(newData3.age.between(18,80))
newData4.show()

+---+------+----+
|age|height|Name|
+---+------+----+
| 30|  12.4|Omar|
| 67|  12.4|Luis|
| 67|  12.4|Luis|
| 67|  12.4|Luis|
+---+------+----+



In [ ]:
newData5=newData4.distinct()
newData5.show()

+---+------+----+
|age|height|Name|
+---+------+----+
| 67|  12.4|Luis|
| 30|  12.4|Omar|
+---+------+----+



In [ ]:
from numpy import tile
newData6=newData5.withColumn("name",sf.upper("name"))
newData6.show()

custom_func=sf.udf(lambda s: f"I am: {s.title()}")

newData6=newData6.withColumn("name",custom_func("name"))
newData6.show()

+---+------+----+
|age|height|name|
+---+------+----+
| 67|  12.4|LUIS|
| 30|  12.4|OMAR|
+---+------+----+

+---+------+----------+
|age|height|      name|
+---+------+----------+
| 67|  12.4|I am: Luis|
| 30|  12.4|I am: Omar|
+---+------+----------+



In [ ]:
newData7=newData6.select("name","age","height")
newData7.show()
newData3

+----------+---+------+
|      name|age|height|
+----------+---+------+
|I am: Luis| 67|  12.4|
|I am: Omar| 30|  12.4|
+----------+---+------+



DataFrame[age: int, height: double, Name: string]